In [5]:
import pandas as pd
import numpy as np
import spotipy
import SpotipyWrapper

In [2]:
help(SpotipyWrapper)

Help on module SpotipyWrapper:

NAME
    SpotipyWrapper

CLASSES
    builtins.object
        WrapperClass
    
    class WrapperClass(builtins.object)
     |  Methods defined here:
     |  
     |  __init__(self)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  doAuth(self)
     |      Funtion: Setup and initialization of credentials manager and 
     |      master Spotify object
     |      
     |      Returns: Spotify Object
     |  
     |  getSongsFromPlaylist(self, uri, username, name='')
     |      Function: Get list of songs from certain playlist
     |      
     |      Returns: Dataframe containing [song name, uri]
     |  
     |  getUsersPlaylists(self, username='mikeydays')
     |      Function: Get a list of users playlist names
     |      
     |      Returns: A list of names
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__


In [6]:
wrapper = SpotipyWrapper.WrapperClass()
sp = wrapper.doAuth()

In [7]:

playlistDF = wrapper.getUsersPlaylists('mikeydays')
playlistDF


,name,uri
0,wholesome bangers,52Z4shkqqeHL1AzX1NUgzY
1,Throwback Dingers,6v3WpQVrQbgfpU2EVxm3eW
2,eminem,2Zfrz2ddkRw64NSG3nRsf3
3,not country,3EaIcfySntnbcOWrBAnb96
4,country,0ZB9jG1uyCbs4rnQ1V5ro6


In [8]:
uri = playlistDF[playlistDF['name'] == 'country']['uri'].values[0]

wrapper.getSongsFromPlaylist(uri, 'mikeydays')

,name,uri
0,I Love This Life,0ZB9jG1uyCbs4rnQ1V5ro6
1,Sunday Morning,0ZB9jG1uyCbs4rnQ1V5ro6
2,Sun Daze,0ZB9jG1uyCbs4rnQ1V5ro6
3,"Courtesy Of The Red, White And Blue (The Angry...",0ZB9jG1uyCbs4rnQ1V5ro6
4,Loving You Easy,0ZB9jG1uyCbs4rnQ1V5ro6
...,...,...
379,Joe Bought a Bar,0ZB9jG1uyCbs4rnQ1V5ro6
380,Is Anybody Loving You These Days,0ZB9jG1uyCbs4rnQ1V5ro6
381,Country Folks - Livin' Loud,0ZB9jG1uyCbs4rnQ1V5ro6
382,"Wine, Beer, Whiskey",0ZB9jG1uyCbs4rnQ1V5ro6


In [9]:
sp.audio_analysis('0ZB9jG1uyCbs4rnQ1V5ro6')

HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0ZB9jG1uyCbs4rnQ1V5ro6 returned 404 due to analysis not found


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/audio-analysis/0ZB9jG1uyCbs4rnQ1V5ro6:
 analysis not found, reason: None